# Super resolution
This notebook tries to repeat the result of [this article](https://arxiv.org/pdf/1511.04587.pdf "Accurate Image Super-Resolution Using Very Deep Convolutional Networks")

## Preparation

In [4]:
import numpy as np
import pandas as pd
import matplotlib 
import caffe
from matplotlib import pyplot as plt
%matplotlib inline

### Prepare dataset

In [ ]:
import lmdb
from PIL import Image

in_db = lmdb.open('image-lmdb', map_size=int(1e12))
with in_db.begin(write=True) as in_txn:
    for in_idx, in_ in enumerate(inputs):
        # load image:
        # - as np.uint8 {0, ..., 255}
        # - in BGR (switch from RGB)
        # - in Channel x Height x Width order (switch from H x W x C)
        im = np.array(Image.open(in_)) # or load whatever ndarray you need
        im = im[:,:,::-1]
        im = im.transpose((2,0,1))
        im_dat = caffe.io.array_to_datum(im)
        in_txn.put('{:0>10d}'.format(in_idx), im_dat.SerializeToString())
in_db.close()

### Prepare model

In [ ]:
caffe.set_mode_gpu()

In [7]:
from caffe import layers as L, params as P

def srcnn(lmdb, batch_size, mean_file='abc'):
    n = caffe.NetSpec()
    
    n.data, n.residue = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb, transform_param=dict(mean_file=mean_file), ntop=2)

    n.conv1 = L.Convolution(n.data, kernel_size=3,num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu1 = L.ReLU(n.conv1, in_place=True)
    n.conv2 = L.Convolution(n.relu1, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu2 = L.ReLU(n.conv2, in_place=True)
    n.conv3 = L.Convolution(n.relu2, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu3 = L.ReLU(n.conv3, in_place=True)
    n.conv4 = L.Convolution(n.relu3, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu4 = L.ReLU(n.conv4, in_place=True)
    n.conv5 = L.Convolution(n.relu4, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu5 = L.ReLU(n.conv5, in_place=True)
    n.conv6 = L.Convolution(n.relu5, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu6 = L.ReLU(n.conv6, in_place=True)
    n.conv7 = L.Convolution(n.relu6, kernel_size=3,num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu7 = L.ReLU(n.conv7, in_place=True)
    n.conv8 = L.Convolution(n.relu7, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu8 = L.ReLU(n.conv8, in_place=True)
    n.conv9 = L.Convolution(n.relu8, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu9 = L.ReLU(n.conv9, in_place=True)
    n.conv10 = L.Convolution(n.relu9, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu10 = L.ReLU(n.conv10, in_place=True)
    n.conv11 = L.Convolution(n.relu10, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu11 = L.ReLU(n.conv11, in_place=True)
    n.conv12 = L.Convolution(n.relu11, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu12 = L.ReLU(n.conv12, in_place=True)
    n.conv13 = L.Convolution(n.relu12, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu13 = L.ReLU(n.conv13, in_place=True)
    n.conv14 = L.Convolution(n.relu13, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu14 = L.ReLU(n.conv14, in_place=True)
    n.conv15 = L.Convolution(n.relu14, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu15 = L.ReLU(n.conv15, in_place=True)
    n.conv16 = L.Convolution(n.relu15, kernel_size=3,num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu16 = L.ReLU(n.conv16, in_place=True)
    n.conv17 = L.Convolution(n.relu16, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu17 = L.ReLU(n.conv17, in_place=True)
    n.conv18 = L.Convolution(n.relu17, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu18 = L.ReLU(n.conv18, in_place=True)
    n.conv19 = L.Convolution(n.relu18, kernel_size=3, num_output=64, weight_filler=dict(type='gaussian', std=0.01))
    n.relu19 = L.ReLU(n.conv19, in_place=True)
    n.conv20 = L.Convolution(n.relu19, kernel_size=3, num_output=1, weight_filler=dict(type='gaussian', std=0.01))

    n.loss =  L.Euclidian(n.conv20, n.residue)
    return n.to_proto()

# Important parameters!
training_set_size = 50000
testing_set_size = 10000
train_batch_size = 150
test_batch_size = 100

In [ ]:
with open('srcnn_auto_train.prototxt', 'w+') as f:
    f.write(str(srcnn('srcnn_train', train_batch, 'means/srcnn_train_mean.binaryproto')))
    
with open('srcnn_auto_test.prototxt', 'w+') as f:
    f.write(str(srcnn('srcnn_test', test_batch, 'means/srcnn_test_mean.binaryproto')))

In [ ]:
!cat srcnn_auto_train.prototxt

In [ ]:
!cat srcnn_auto_train.prototxt

In [ ]:
!cat srcnn_auto_solver.prototxt

In [ ]:
# initialize solver
solver = caffe.SGDSolver('srcnn_auto_solver.prototxt')

In [ ]:
# restore model from iteration x
solver.net.copy_from('breakpoints/toX.caffemodel')

In [ ]:
# define the structure of the model for the first time
model_def = 'task2/alexnet_auto_train.prototxt'
net = caffe.Net(model_def,
                caffe.TRAIN)

In [ ]:
# each output is (batch size, feature dim, spatial dim)
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

In [ ]:
# just print the weight sizes (we'll omit the biases)
[(k, v[0].data.shape) for k, v in solver.net.params.items()]

## Work

In [ ]:
%%time

plt.figure(num=1, figsize=(48,24), dpi=80, facecolor='w', edgecolor='k')
plt.subplot()

niter = 20000
test_interval = 500
# losses will also be stored in the log
train_loss = np.zeros(niter)
test_loss = np.zeros(int(np.ceil(niter / test_interval)))
pictures_number = niter / test_interval
rowsnum = 4
picidx = 0
# the main solver loop
for it in range(niter):
    solver.step(1)  # SGD by Caffe
    # store the train loss
    train_loss[it] = solver.net.blobs['loss'].data
    
    # run a full test every so often
    # (Caffe can also do this for us and write to a log, but we show here
    #  how to do it directly in Python, where more complicated things are easier.)
    if it % test_interval == 0:
        picidx += 1
        print 'Iteration', it, 'testing...'
        score = 0.0
        for test_it in range(test_set_size / test_batch_size):
            solver.test_nets[0].forward()
            score = solver.test_nets[0].blobs['loss'].data
        test_loss[it / test_interval] = score
        
        # plt middle results
        plt.subplot(rowsnum, pictures_number/rowsnum, picidx)
        plt.grid(True)
        plt.plot(np.arange(niter), train_loss)
        plt.plot(test_interval * np.arange(len(test_score)), test_acc, 'r')
        plt.xlabel('iteration')
        plt.ylabel('loss')
        plt.title('loss, iteration %d' % it)
        plt.legend(['train', 'test'], loc='best')
        plt.show()

## Other

In [ ]:
# send results via email
#!/usr/bin/env python
# encoding: utf-8

import smtplib
from datetime import datetime


def noticeEMail(train_str, test_str, usr, psw, fromaddr, toaddr):
    """
    Sends an email message through GMail once the script is completed.  
    Developed to be used with AWS so that instances can be terminated 
    once a long job is done. Only works for those with GMail accounts.
    
    starttime : a datetime() object for when to start run time clock
    usr : the GMail username, as a string
    psw : the GMail password, as a string 
    
    fromaddr : the email address the message will be from, as a string
    
    toaddr : a email address, or a list of addresses, to send the 
             message to
    """

    
    # Initialize SMTP server
    server=smtplib.SMTP('smtp.gmail.com:587')
    server.starttls()
    server.login(usr,psw)
    
    # Send email
    senddate=datetime.strftime(datetime.now(), '%Y-%m-%d')
    subject="Your job is complete"
    m="Date: %s\r\nFrom: %s\r\nTo: %s\r\nSubject: %s\r\nX-Mailer: My-Mail\r\n\r\n" % (senddate, fromaddr, toaddr, subject)
    msg='''
    
    train: ''' + train_str + ''' 
    
    test ''' + test_str
    
    server.sendmail(fromaddr, toaddr, m+msg)
    server.quit()


if __name__ == '__main__':    
    # Fill these in with the appropriate info...
    usr='dem4064@gmail.com'
    psw=''
    fromaddr='dem4064@gmail.com'
    toaddr='dmitriy.denisenko@phystech.edu'

    # Send notification email
    noticeEMail(np.array_str(train_loss), np.array_str(test_loss), usr, psw, fromaddr, toaddr)